In [68]:
# -*- coding: utf-8 -*-
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta,date
import locale
import re 
import os 
from pathlib import Path
import pandas as pd
import time
import random 
import sys
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

'en_US.UTF-8'

In [2]:

def get_followers_id(soup,url_str='https://weibo.cn'):
    user_names = [a.get_text() for a in soup.find_all('a', href=True) if url_str in a['href'] and a.get_text()]
    user_uids = [a['href'].split('/')[-1] for a in soup.find_all('a', href=True) if url_str in a['href'] and a.get_text()]
    user_uids = [item for item in user_uids if not '=' in item or '.' in item]
    return user_names, user_uids

def save_array2txt(my_array,filename):
    # Open the file for writing
    with open(filename, 'w') as file:
        for item in my_array:
        # Write the string representation of the array to the file
            file.write(item)
            file.write('\n')
    return True

def random_wait(min_seconds=15,max_seconds=30):
    random_seconds = random.randint(min_seconds, max_seconds)
    time.sleep(random_seconds)
    print(f'get next page after {random_seconds} seconds')
    return True

In [ ]:


with open('cookies.txt', 'r', encoding='utf-8') as f:
    cookie = f.read().strip()

# 设置请求头部信息
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'Cookie': cookie
}

# 定义需要抓取关注列表的微博用户ID和页数
user_id = '6235235201'
page = 1
user_names = []
user_uids = []


profile_url = f'https://weibo.cn/{user_id}/profile'
profile_response = requests.get(profile_url, headers=headers)
profile_soup = BeautifulSoup(profile_response.content, 'html.parser')
user_name = profile_soup.find('title').text.split(' ')[0][:-3]
print(user_name)



# 构造请求URL
#url = f'https://weibo.cn/u/{user_id}?page={page}'
url = f'https://weibo.cn/{user_id}/follow?page={page}'
# 发送请求
response = requests.get(url, headers=headers)
random_wait()

# 解析HTML页面并提取关注列表
soup = BeautifulSoup(response.content, 'html.parser') #html5lib  
followings = get_followers_id(soup)
user_names += followings[0]
user_uids += followings[1]

random_wait()

page_next = soup.find('a', string='下页')
while page_next:
    # 获取下一页的URL
    page += 1
    url = f'https://weibo.cn/{user_id}/follow?page={page}'
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    followings = get_followers_id(soup)
    user_names += followings[0]
    user_uids += followings[1]
    page_next = soup.find('a', string='下页')
    random_wait()

user_names = list(set(user_names))
user_uids = list(set(user_uids))

save_array2txt(user_names,f'./following_lists/{user_id}_{user_name}_followers_names.txt')
save_array2txt(user_uids,f'./following_lists/{user_id}_{user_name}_followers_uids.txt')

In [89]:
def standardize_date(created_at):
        """标准化微博发布时间"""
        if "刚刚" in created_at:
            ts = datetime.now()
        elif "分钟" in created_at:
            #print('find 分钟')
            minute = created_at[: created_at.find("分钟")]
            minute = timedelta(minutes=int(minute))
            ts = datetime.now() - minute
        elif "小时" in created_at:
            #print('find 小时')
            hour = created_at[: created_at.find("小时")]
            hour = timedelta(hours=int(hour))
            ts = datetime.now() - hour
        elif "今天" in created_at:  
            minute = created_at[3:8]
            #print(f'find 今天 {minute}') 
            date_str = datetime.now().strftime('%Y-%m-%d') 
            ts = datetime.strptime(f'{date_str} {minute}','%Y-%m-%d %H:%M')
        elif "昨天" in created_at:
            #print('find 昨天')
            day = timedelta(days=1)
            ts = datetime.now() - day
        else:
            #print('none')
            date_str,time_str = created_at.split(' ')[0],created_at.split(' ')[1]
            time_str = time_str[:5]
            year_str = str(datetime.now().year)
            ts = datetime.strptime(f'{year_str}年{date_str} {time_str}','%Y年%m月%d日 %H:%M')

        return ts

def standardize_info(weibo):
    """标准化信息，去除乱码"""
    for k, v in weibo.items():
        if (
            "bool" not in str(type(v))
            and "int" not in str(type(v))
            and "list" not in str(type(v))
            and "long" not in str(type(v))
        ):
            weibo[k] = (
                v.replace("\u200b", "")
                .encode(sys.stdout.encoding, "ignore")
                .decode(sys.stdout.encoding)
            )
    return weibo

def download_image(url, filename):
    """
    下载图片
    """
    filename = Path(filename)
    filename.parent.mkdir(parents=True, exist_ok=True)

    r = requests.get(url)
    if r.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(r.content)

def download_video(url, path):
    """
    下载视频
    """
    if not os.path.exists(path):
        os.makedirs(path)
    r = requests.get(url, stream=True)
    if r.status_code == 200:
        with open(os.path.join(path, os.path.basename(url)), 'wb') as f:
            for chunk in r.iter_content(1024*1024):
                f.write(chunk)

def get_comments(comment_url,headers):
    comment_response = requests.get(comment_url, headers=headers)
    comment_soup = BeautifulSoup(comment_response.content, 'html.parser')
    comment_text = [a.get_text() for a in comment_soup.find_all('span', class_='ctt') if a.get_text()]
    comment_count_get = len(comment_text)#int(comment_text.split('[')[-1].split(']')[0])
    return comment_count_get,comment_text[1:]
     

def get_page_content(user_id,page,headers,curr_time,time_limit,user_folder,original_pic_download=False):
    next_page = True
    # 构造请求URL
    url = f'https://weibo.cn/{user_id}?filter=1&page={page}'

    # 发送请求
    response = requests.get(url, headers=headers)

    # 解析HTML页面并提取原创微博
    soup = BeautifulSoup(response.content, 'html.parser')
    weibos = soup.find_all('div', class_='c', id=True)

    weibo_content = []

    # 输出原创微博
    for weibo in weibos:
        # 提取微博发布时间
        create_time_str = weibo.find('span', class_='ct').get_text()
        print(f'create_at: {create_time_str}')
        create_at = standardize_date(create_time_str)
        if create_at > time_limit:
            # 提取微博正文
            content = weibo.find('span', class_='ctt').get_text()
            comment_count_text = weibo.find('a',href=True,class_='cc').get_text()
            comment_count = re.search(r'\[(\d+)\]', comment_count_text).group(1)
            comment_url = weibo.find('a',href=True,class_='cc')['href']
            comment_count_get,comment_text = get_comments(comment_url,headers)

            weibo_content.append({'create_at':create_at,'content':content,'comment_count':comment_count,'comment_count_get':comment_count_get,'comment_text':comment_text})
            print(content)
            if original_pic_download:
                imgs = weibo.find_all('img', alt='图片')
                for i,img in enumerate(imgs):
                    img_url = img.get('src')
                    image_folder = Path(os.path.join(user_folder,'img'))
                    image_folder.mkdir(parents=True,exist_ok=True)
                    file_name = os.path.join(image_folder,f'{user_id}_{time.strftime("%Y%m%d%H%M%S")}_{i}.jpg')
                    download_image(img_url, file_name)

        else:
            print('get last weibo before: ',create_at)
            next_page = False
            break
        page_next = soup.find('a', string='下页')
        if not len(page_next):
            next_page = False
    return next_page,weibo_content
    


In [90]:
# 设置请求头部信息和Cookie
with open('cookies.txt', 'r', encoding='utf-8') as f:
    cookie = f.read().strip()

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'Cookie': cookie
}

# 定义需要抓取的微博用户ID和页数
user_id = '6028429521'
original_pic_download=True
page = 1
hours = 2
next_page = True

profile_url = f'https://weibo.cn/{user_id}/profile'
profile_response = requests.get(profile_url, headers=headers)
profile_soup = BeautifulSoup(profile_response.content, 'html.parser')
user_name = profile_soup.find('title').text.split(' ')[0][:-3]
print(user_name)
random_wait()

user_folder = Path(os.path.join(os.getcwd(),'weibo',user_name))
user_folder.mkdir(parents=True,exist_ok=True)

# 计算最近n小时的时间
curr_time = datetime.now()
time_limit = curr_time - timedelta(hours=hours)
print('get weibos until',time_limit.strftime('%m月%d日 %H:%M'))
today_str = time_limit.strftime('%m月%d日')
all_weibo_content = []
next_page,weibo_content = get_page_content(user_id,page,headers,curr_time,time_limit,user_folder)
all_weibo_content.extend(weibo_content)
random_wait()

while(next_page):
    # 获取下一页的URL
    page += 1
    print(f'get content from page {page}')
    next_page,weibo_content = get_page_content(user_id,page,headers,curr_time,time_limit,user_folder)
    if next_page==False:
        break
    all_weibo_content.extend(weibo_content)
    print(f'find next page {next_page}')
    random_wait()

    

茶美丽的追剧日常
get next page after 10 seconds
get weibos until 03月18日 09:14
create_at: 3分钟前 来自沦落民间的iPhone 13(粉色)
CPB才是今天真流量的试金石吧？首先这个牌子8紫都没代言，比较客观。其次这个牌子是蔡顶流代言，捍卫本属于自己的local非常合理。就结果来说：第一依旧是肖顶流，这个没什么疑问，蔡顶流只要拿到第二，我认为就算可以了，你们能接受我这个标准吗？ 
create_at: 12分钟前 来自沦落民间的iPhone 13(粉色)
男艺人签到人数前五：肖战、王一博、易烊千玺、王俊凯、龚俊女艺人签到人数前三：刘雨昕、迪丽热巴、鞠婧祎这个签到是不是能看出真实的粉丝体量？ 
create_at: 14分钟前 来自沦落民间的iPhone 13(粉色)
#迪丽热巴流光闪烁花絮# 来看看迪丽热巴超近距离直拍，那个甩头发回眸一笑，路人都尖叫了 
create_at: 43分钟前 来自沦落民间的iPhone 13(粉色)
#杨洋白西服#杨洋的妆造，always给我一种敷衍的赶脚，always白色系，他真的好爱白色。咋说呢？白色系有的时候太像男g关了，会把他本身的贵气压制，时尚圈一般是很少用一身白色系的，杨洋对白色系的执迷，我表示真滴很：敬佩。 
create_at: 45分钟前 来自沦落民间的iPhone 13(粉色)
#人生路遥改名人生之路#李沁的大青衣之路，即将开启！她后面还有和肖战合作的主流剧《梦中的那片海》、和王一博合作的主流剧《光明之路》，可以说流量+主流双保险！后续弯道超车，不是没可能的！ 
create_at: 48分钟前 来自沦落民间的iPhone 13(粉色)
#迪丽热巴流光闪烁花絮#这就是所谓的缘分吧[滑稽] 
create_at: 51分钟前 来自沦落民间的iPhone 13(粉色)
#光渊#听说这个剧被点名批评了啊但播还是可以播的。 
create_at: 今天 09:32 来自沦落民间的iPhone 13(粉色)
蔡徐坤这个大C位是不是拿的也不是多光彩。其他人没什么咖的赶脚。 
create_at: 今天 09:31 来自沦落民间的iPhone 13(粉色)
纯甄热转数据统计前十杨紫是唯一的女艺人是不是因为她代言这个，所以爱丽丝不想给她代言的产品贡献热度？ 
c

In [91]:
content_df = pd.DataFrame(all_weibo_content)
content_df['user_uid'] = user_id
content_df['user_name'] = user_name
content_df['last_check'] = time_limit.strftime('%Y-%m-%d %H:%M')
content_df['check_before'] = hours
content_df

,create_at,content,comment_count,comment_count_get,comment_text,user_uid,user_name,last_check,check_before
0,2023-03-18 11:11:56.183435,CPB才是今天真流量的试金石吧？首先这个牌子8紫都没代言，比较客观。其次这个牌子是蔡顶流代言...,4,5,"[铁粉1就很难压了其实，还没试过金粉牌什么感觉[欢乐熊出没], 别急，等着看热闹！看心情交吃...",6028429521,茶美丽的追剧日常,2023-03-18 09:14,2
1,2023-03-18 11:02:56.681155,男艺人签到人数前五：肖战、王一博、易烊千玺、王俊凯、龚俊女艺人签到人数前三：刘雨昕、迪丽热巴...,22,11,"[gj就算了吧，他最氵，生日月签到比其他时间多了20万, 不一定，听说这个也能买，我记得以前...",6028429521,茶美丽的追剧日常,2023-03-18 09:14,2
2,2023-03-18 11:00:57.315615,#迪丽热巴流光闪烁花絮# 来看看迪丽热巴超近距离直拍，那个甩头发回眸一笑，路人都尖叫了,9,10,"[确实漂亮，就是把美女冻着了，我看着都发抖, 太美了，很難忍住不尖叫[送花花], 生图都美翻...",6028429521,茶美丽的追剧日常,2023-03-18 09:14,2
3,2023-03-18 10:31:57.794744,#杨洋白西服#杨洋的妆造，always给我一种敷衍的赶脚，always白色系，他真的好爱白色...,35,12,[品牌活动，跟杨洋有啥关系，没话题就别硬蹭，杨洋想穿什么颜色穿什么颜色，他穿啥都好看。 评论...,6028429521,茶美丽的追剧日常,2023-03-18 09:14,2
4,2023-03-18 10:29:58.576493,#人生路遥改名人生之路#李沁的大青衣之路，即将开启！她后面还有和肖战合作的主流剧《梦中的那片...,19,13,"[你男朋友也出演了, 她又不是一番，肖燕一番收视率1.5，不是照样还是三线演员, 你想多了，...",6028429521,茶美丽的追剧日常,2023-03-18 09:14,2
5,2023-03-18 10:26:59.321782,#迪丽热巴流光闪烁花絮#这就是所谓的缘分吧[滑稽],6,7,"[回复@g-junya201909:紫设计师粉丝画的礼服画稿这个气质…, 迪丽热巴独美 评论...",6028429521,茶美丽的追剧日常,2023-03-18 09:14,2
6,2023-03-18 10:23:59.869476,#光渊#听说这个剧被点名批评了啊但播还是可以播的。,14,14,"[拉倒吧，真不让播还能预热几天，自己炒作还要甩锅, 这剧还有人看？, 拉到, 回复@山月不知...",6028429521,茶美丽的追剧日常,2023-03-18 09:14,2
7,2023-03-18 09:32:00.000000,蔡徐坤这个大C位是不是拿的也不是多光彩。其他人没什么咖的赶脚。,76,13,"[你就说这几个人里面，哪个不比他能力强？, 他旁边的那几位在歌手里咖位不低啊。容祖儿算是新香...",6028429521,茶美丽的追剧日常,2023-03-18 09:14,2
8,2023-03-18 09:31:00.000000,纯甄热转数据统计前十杨紫是唯一的女艺人是不是因为她代言这个，所以爱丽丝不想给她代言的产品贡献...,62,13,[一个简单的逻辑问题，热转100万+，如果真有这么多人，不可能一个热评都进不去，之所以一个都...,6028429521,茶美丽的追剧日常,2023-03-18 09:14,2
9,2023-03-18 09:19:00.000000,扁桃体发炎了🙃有没有推荐的药🙃,62,11,"[蒲地蓝, 你昨天太挑事了，回旋镖了呗, 扁桃体就用抗生素效果最好，不化脓不要用喷剂，基本上...",6028429521,茶美丽的追剧日常,2023-03-18 09:14,2


In [92]:

def contains_chinese_keywords(text, keywords):
    query_result = []
    for keyword in keywords:
        # 使用正则表达式查找中文关键词
        pattern = re.compile(keyword)
        match = pattern.search(text)
        if match:
            query_result.append(keyword)
    return query_result



In [93]:
keywords = ["成毅"]
for index,row in content_df.iterrows():
    query_keywords_result = contains_chinese_keywords(row['content'],keywords)
    if len(query_keywords_result):
        print(row['create_at'],row['content'])